In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100

In [3]:
df_1 = pd.read_csv(r"C:\Users\User\Documents\GitHub\Iron-Moran-\Data_Marketing_Customer_Analysis_Round3.csv")
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10689 entries, 0 to 10688
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   region                         10689 non-null  object
 1   customer_lifetime_value        10689 non-null  int64 
 2   response                       10689 non-null  object
 3   coverage                       10689 non-null  object
 4   education                      10689 non-null  object
 5   effective_to_date              10689 non-null  object
 6   month                          10689 non-null  object
 7   employment_status              10689 non-null  object
 8   gender                         10689 non-null  object
 9   income                         10689 non-null  int64 
 10  location_code                  10689 non-null  object
 11  marital_status                 10689 non-null  object
 12  monthly_premium_auto           10689 non-null  int64 
 13  m

In [4]:
data = df_1.select_dtypes(include='int64')

In [5]:
X = data.drop(columns=['total_claim_amount'], axis = 1)
y = data['total_claim_amount']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test  = pd.DataFrame(X_test, columns=X.columns)

In [7]:
X_train.describe()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
count,8551.000000,8551.000000,8551.000000,8551.00000,8551.000000,8551.000000,8551.000000
mean,7994.902701,51817.509063,93.295287,15.13554,48.192960,0.375395,2.983511
std,6848.846659,24717.379264,34.575537,10.13316,27.849503,0.899706,2.398456
min,1898.000000,10074.000000,61.000000,0.00000,0.000000,0.000000,1.000000
25%,4020.500000,29435.000000,68.000000,6.00000,25.000000,0.000000,1.000000
50%,5764.000000,50446.000000,83.000000,14.00000,48.000000,0.000000,2.000000
75%,8964.000000,72194.500000,109.000000,23.00000,71.000000,0.000000,4.000000
max,74228.000000,99981.000000,298.000000,35.00000,99.000000,5.000000,9.000000


In [8]:
from sklearn.feature_selection import VarianceThreshold # It only works with numerical features


X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

#display(X_train)
print("Initial number of numerical columns: ",X_train.shape)
print()


selector = VarianceThreshold(threshold=100) # Default threshold value is 0
# Features with a training-set variance lower than this threshold will be removed.
selector.fit(X_train)

kept_features_indexes = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features_indexes].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final number of numerical columns: ",X_train.shape)
print()
X_train

Initial number of numerical columns:  (8551, 7)

Final number of numerical columns:  (8551, 5)



,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception
0,21423,22379,65,9,31
1,8391,40211,106,5,98
2,3969,49544,101,3,29
3,14914,45963,63,3,73
4,18060,57882,115,1,61
...,...,...,...,...,...
8546,7610,98701,94,22,66
8547,35186,86134,98,17,78
8548,4241,19834,64,26,8
8549,12941,77060,106,23,90


In [9]:
import seaborn as sns
import matplotlib.pyplot as plt

c = abs(data.corr())
#c

#fig, ax = plt.subplots(figsize=(14,14))
#sns.heatmap(c, annot=True);

#c['SalePrice']
c_last = c['total_claim_amount'].sort_values(ascending=False)
#c_last
c_thr = .3
cols_to_keep = list(c_last[c_last > c_thr].index)[1:] + [list(c_last[c_last > c_thr].index)[0]]
print(cols_to_keep)

data[cols_to_keep]

['monthly_premium_auto', 'total_claim_amount']


,monthly_premium_auto,total_claim_amount
0,61,292
1,64,744
2,100,480
3,97,484
4,117,707
...,...,...
10684,253,1214
10685,65,273
10686,201,381
10687,158,618


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE  ## recursive feature elemination technique

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test  = pd.DataFrame(X_test, columns=X.columns)

#X_train.isna().sum()
nulls = pd.DataFrame(X_train.isna().sum()).reset_index()
#nulls.head()
nulls.columns = ['Column','nas']
#nulls.head()
#nulls[nulls['nas'] > 0].head()
cols_to_drop = nulls[nulls['nas'] > 0]['Column'] # Too drastic, but made on pourpose for quick filtering (don't do this in production!!)

X_train.drop(columns=cols_to_drop, axis=1, inplace = True)
X_test.drop(columns=cols_to_drop, axis=1, inplace = True)

#display(X_train)

lm = LinearRegression()

selector = RFE(lm, n_features_to_select= 2, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final selected features: ")
display(X_train)

Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Final selected features: 


,monthly_premium_auto,number_of_policies
0,65,2
1,106,6
2,101,1
3,63,2
4,115,2
...,...,...
8546,94,3
8547,98,2
8548,64,8
8549,106,2


##  Embedded Methods

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

In [25]:
import numpy as np
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train=imp_mean.fit_transform(X_train)
X_test=imp_mean.fit_transform(X_test)

In [26]:
X_train

array([[8.6630e+03, 4.2169e+04, 8.3000e+01, ..., 9.0000e+01, 1.0000e+00,
        2.0000e+00],
       [4.2130e+03, 1.2160e+04, 1.0900e+02, ..., 3.4000e+01, 0.0000e+00,
        1.0000e+00],
       [2.3590e+03, 1.9864e+04, 6.3000e+01, ..., 9.6000e+01, 0.0000e+00,
        1.0000e+00],
       ...,
       [4.2410e+03, 1.9834e+04, 6.4000e+01, ..., 8.0000e+00, 4.0000e+00,
        8.0000e+00],
       [1.2941e+04, 7.7060e+04, 1.0600e+02, ..., 9.0000e+01, 0.0000e+00,
        2.0000e+00],
       [6.9470e+03, 6.3406e+04, 1.0000e+02, ..., 5.4000e+01, 0.0000e+00,
        7.0000e+00]])

## OLS

In [27]:
model=LinearRegression()
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

LinearRegression: Train -> 0.4086926440650056, Test -> 0.4114006217055415


# lasso

In [28]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
model=Lasso(alpha=0.05)

model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

Lasso: Train -> 0.4086926032425867, Test -> 0.41141620918570554


# RIDGE

In [31]:
model=Ridge(alpha=10000)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

Ridge: Train -> 0.40867692596088767, Test -> 0.41168523050262207


# Fitting OLS

In [11]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from sklearn.datasets import load_boston

In [13]:
data

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,4809,48029,61,7,52,0,9,292
1,2228,92260,64,3,26,0,1,744
2,14947,22139,100,34,31,0,2,480
3,22332,49078,97,10,3,0,2,484
4,9025,23675,117,33,31,0,7,707
...,...,...,...,...,...,...,...,...
10684,15563,61541,253,12,40,0,7,1214
10685,5259,61146,65,7,68,0,6,273
10686,23893,39837,201,11,63,0,2,381
10687,11971,64195,158,0,27,4,6,618


In [16]:
x = data.copy()

y = x.total_claim_amount
X = pd.DataFrame(x, columns = x.columns)

X_added_constant = sm.add_constant(X)
X_added_constant

,const,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1.0,4809,48029,61,7,52,0,9,292
1,1.0,2228,92260,64,3,26,0,1,744
2,1.0,14947,22139,100,34,31,0,2,480
3,1.0,22332,49078,97,10,3,0,2,484
4,1.0,9025,23675,117,33,31,0,7,707
...,...,...,...,...,...,...,...,...,...
10684,1.0,15563,61541,253,12,40,0,7,1214
10685,1.0,5259,61146,65,7,68,0,6,273
10686,1.0,23893,39837,201,11,63,0,2,381
10687,1.0,11971,64195,158,0,27,4,6,618
